<a href="https://colab.research.google.com/github/cserock/colab-examples/blob/main/04_CNN_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

10개의 클래스로 분류되는 이미지 데이터셋인 [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)을 사용하여 CNN을 어떻게 구현하는지 알아보자.

1. 패키지 및 라이브러리 불러오기

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

2. 시드(seed) 고정하기

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

3. 데이터셋 불러오기 & 훈련 / 검증 / 테스트셋으로 나누기 & 이미지 픽셀값 정규화

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2)

In [ ]:
X_train.shape

(40000, 32, 32, 3)

훈련 데이터셋으로 가로 32, 세로 32 픽셀의 컬러(RGB, 3개 채널) 이미지 40000개가 있다.

4. 합성곱 신경망 모델 구축

In [ ]:
model = Sequential()

# 특징 추출 부분
# 합성곱 층(Conv2D)와 풀링 층(MaxPooling2D)를 번갈아가며 사용
# input_shape 파라미터 추가 - 예: CIFAR-10 데이터셋 (32x32x3)
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2,2))

# 분류기 역할의 완전 연결 신경망
# 특징 추출 부분을 거쳐온 데이터를 신경망에 입력할 수 있도록 1차원으로 변환
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# 10개의 클래스 분류이므로 출력층에 10개의 노드를, 활성화 함수로 softmax를 지정
model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Conv2D에서 반드시 지정해야 하는 첫 번째 파라미터는 필터의 수(filters), 두 번째는 필터(커널)의 크기(kernel_size)이다.  
padding은 'valid' 또는 'same'로 지정 가능하다.  
'valid'는 패딩을 적용하지 않아 Conv2D를 지나면 입력된 이미지의 shape이 작아진다.  
'same'은 패딩을 적용하여 Conv2D 전후의 이미지 shape이 동일하게 만든다.  
  
MaxPooling2D에서 풀링할 영역의 크기(pool_size)는 (2, 2)가 기본값이다.  
strides 파라미터를 지정하여 몇 칸 단위로 움직이며 풀링을 할지 지정할 수 있다.   기본값은 None으로, 이대로 두면 pool_size와 동일하게 설정된다. 즉, 풀링 영역이 겹치지 않게 된다.

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,346 (337.29 KB)

 Trainable params: 86,346 (337.29 KB)

 Non-trainable params: 0 (0.00 B)

Output Shape은 (행 수, 가로 픽셀 수, 세로 픽셀 수, 채널 수) 라고 생각하면 쉽다.  
여기서 행(row) 수는 None으로 표시되어 있는데, 이는 특정한 숫자로 지정되지 않았음을 의미한다. 배치 사이즈처럼 32, 64 등 다양한 숫자가 올 수도 있기 때문이다.  
채널 수는 각 Conv2D에서 지정한 filters의 값에 맞춰 나온 것을 볼 수 있다.  
  
MaxPooling2D는 입력으로 들어온 이전의 Conv2D의 shape을 줄였다.  
채널의 수는 입력 데이터와 동일하며, 학습되는 가중치가 없기 때문에 Param #도 0으로 표기되었다.  
  
Flatten에서는 (4, 4, 32)의 데이터가 1차원으로 변환되었기 때문에 4*4*32=512의 shape을 갖게 되었다.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

10개 클래스의 다중 분류 문제이므로 'loss'를 'sparse_categorical_crossentropy'로 지정했다.

In [ ]:
model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 57s 173ms/step - accuracy: 0.3027 - loss: 1.9027 - val_accuracy: 0.4876 - val_loss: 1.4241
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 170ms/step - accuracy: 0.5016 - loss: 1.3836 - val_accuracy: 0.5436 - val_loss: 1.2826
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 175ms/step - accuracy: 0.5584 - loss: 1.2380 - val_accuracy: 0.5813 - val_loss: 1.1733
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 80s 171ms/step - accuracy: 0.6006 - loss: 1.1303 - val_accuracy: 0.6187 - val_loss: 1.0859
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 58s 184ms/step - accuracy: 0.6283 - loss: 1.0478 - val_accuracy: 0.6380 - val_loss: 1.0388
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 77s 169ms/step - accuracy: 0.6544 - loss: 0.9780 - val_accuracy: 0.6547 - val_loss: 0.9824
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 176ms/step - accuracy: 0.6770 - loss: 0.9208 - val_accuracy: 0.6637 - val_loss: 0.9598
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 56s 180ms/step - accuracy: 0.6935 - loss: 0

5. 모델 평가

In [ ]:
model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - 4ms/step - accuracy: 0.6815 - loss: 0.9356


[0.9355661869049072, 0.6815000176429749]

# 전이 학습

위의 CIFAR-10 데이터셋을 그대로 사용하되,  
모델은 사전 학습 모델(VGG16)을 이용한 전이 학습 모델로 구축

패키지 및 라이브러리 불러오기

In [19]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

사전 학습 모델(VGG16) 불러오기  
- weights='imagenet' : 사전 학습된 가중치를 ImageNet 데이터셋으로부터 불러옴  
- include_top=False : 최종 분류층(완전 연결 계층들)을 포함하지 않고 특징 추출용 특성 맵만 반환하도록 설정

In [20]:
pretrained_model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


사전 학습 모델 위에 분류기 추가하기  
아래에 추가된 GlobalAveragePooling2d() 층은 데이터 Shape을 (None, None, None, 512) 에서 (None, 512)로 변환하는 역할을 한다.

In [21]:
model = Sequential()
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [22]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, None, None,     │    14,714,688 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,781,642 (56.39 MB)

 Trainable params: 14,781,642 (56.39 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 50s 121ms/step - accuracy: 0.1551 - loss: 2.2058 - val_accuracy: 0.2972 - val_loss: 1.6837
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.3392 - loss: 1.6150 - val_accuracy: 0.4674 - val_loss: 1.3805
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.5501 - loss: 1.2103 - val_accuracy: 0.6474 - val_loss: 1.0120
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - accuracy: 0.6823 - loss: 0.9183 - val_accuracy: 0.6957 - val_loss: 0.8784
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.7380 - loss: 0.7753 - val_accuracy: 0.7094 - val_loss: 0.8756
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.7874 - loss: 0.6411 - val_accuracy: 0.7373 - val_loss: 0.8181
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.8130 - loss: 0.5679 - val_accuracy: 0.7351 - val_loss: 0.8820
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.8362 - loss: 0.4996 -

In [25]:
model.evaluate(X_test, y_test, verbose=2)

313/313 - 4s - 13ms/step - accuracy: 0.7669 - loss: 0.7831


[0.7830758690834045, 0.7669000029563904]